In [36]:
# SETUP
import pandas as pd
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import re
import time

# import requests

In [37]:

# URLS
shein_home_page_url="https://eur.shein.com/"

shein_clothing_url_format="https://eur.shein.com/Clothing-c-2035.html?ici=eur_tab01navbar04&scici=navbar_WomenHomePage~~tab01navbar04~~4~~webLink~~~~0~~50001&page={0}"

shein_clothing_url="https://eur.shein.com/Clothing-c-2035.html?ici=eur_tab01navbar04&scici=navbar_WomenHomePage~~tab01navbar04~~4~~webLink~~~~0~~50001"
#CONSTS

columns_list=['SKU',
                'Item Name',
                'Price',
                'Reviews Num',
                '1 Star Reviews',
                '1 Star Images',
                '2 Stars Reviews',
                '2 Stars Images',
                '3 Stars Reviews',
                '3 Stars Images',
                '4 Stars Reviews',
                '4 Stars Images',
                '5 Stars Reviews',
                '5 Stars Images',
                'Points Earnning',
                'Style',
                'Color',
                'Pattern Type',
                'Length',
                'Type',
                'Neckline',
                'Sleeve Length',
                'Sleeve Type',
                'Placket Type',
                'Details',
                'Season',
                'Composition',
                'Material',
                'Fabric',
                'Sheer',
                'Fit Type',
                'Closure Type',
                'Waist Line',
                'Panty Type',
                'Body',
                'Chest pad',
                'Bra Type',
                'Bottom Type']


In [38]:
# bs4 is not supporting dinamic data loading, so I had to use selenium web driver 
# to get the linked javascript to the html's.   
driver = webdriver.Chrome(executable_path=r"C:\Users\ofirr\Documents\שנה ב'\מבוא למדעי הנתונים\Fashion project\crawling\chromedriver.exe")

In [39]:
#the function returns the number of pages in the clothings section. 
def pages_amount():
    driver.get(shein_clothing_url)
    html=driver.page_source
    soup=BeautifulSoup(html)
    info=soup.find("span", {"class": "S-pagination__total"})
    pages=info.text
    pattern=r"Total (\d+) Pages"
    return (int(re.findall(pattern,pages)[0]))


In [40]:
# the function returns all of the items links from single page (1-40 pages)
def get_items_links(shein_page_url):
    links=[]
    colord_item_format="https://eur.shein.com/{0}-p-{1}-cat-1773.html"
    single_item_format="https://eur.shein.com{0}"
    driver.get(shein_page_url)
    html=driver.page_source
    soup=BeautifulSoup(html)
    items_info=soup.findAll("div", {"class": "S-product-item__info"})
    for item_info in items_info:
        color_container=item_info.find("section",{"class": "S-product-item__relatecolor-container"})
        if(color_container is not None):
            for lable in color_container.findAll("label"):
                aria=(lable.get("aria-label"))
                value=(lable.input.get("value"))
                aria=aria.replace(" ","-")
                str1=colord_item_format.format(aria,value)
                links.append(str1)
                
        else: 
            str2=single_item_format.format(item_info.a.get("href"))
            links.append(str2)
    return links

In [41]:
def get_url_for_each_page(pages_count):
    all_products_links=[]
    for i in range(28,pages_count):
        curr_url=shein_clothing_url_format.format(i)
        all_products_links.extend(get_items_links(curr_url)) #appending all lists together from each page
    return all_products_links

In [42]:
def get_ranks_level(driver,star):
    try:
        drop_down = driver.find_element_by_class_name('common-reviews__select-box-list')
        drop_box = driver.find_element_by_class_name('common-reviews__select-box-input')
        lines=drop_down.find_elements_by_tag_name('li')
        hover_action = ActionChains(driver).move_to_element(drop_box)
        hover_action.perform()
        lines[5-star].click()
        time.sleep(1)
        all_reviews=driver.find_element_by_class_name("j-expose__review-tab-container")
        all_reviews_text=all_reviews.text
        pattern=r'All Reviews (\(\w+)\+?\)'
        all_reviews_value=int((re.findall(pattern,all_reviews_text)[0])[1:])
        images=driver.find_element_by_class_name("j-expose__review-image-tab-target")
        images_text=images.text
        pattern=r'Image (\(\d+\))'
        images_value=int((re.findall(pattern,images_text)[0])[1:-1])
        return (all_reviews_value,images_value)
    except:
        return(0,0)

In [43]:
def get_details_from_single_item(item_link):
    description_dictionary={}
    driver.get(item_link)
    html=driver.page_source
    soup=BeautifulSoup(html)
    
    #SKU
    SKU=soup.find("div",{"class": "product-intro__head-sku"})
    SKU_text=SKU.text
    pattern=r"SKU: (\w+)"
    SKU_value=re.findall(pattern,SKU_text)[0]
    
    #Item Name
    
    Item_Name=soup.find("div",{"class": "product-intro__head-name"})
    Item_Name_text=Item_Name.text
    pattern=r"\n\s+(.*)\n"
    Item_Name_value=re.findall(pattern,Item_Name_text)[0]
    
    #Price
    price=soup.find("div",{"class": "from"})
    price_text=price.span.text
    price_text=price_text.replace(',','.')
    price_value=float(price_text[0:-1])
    
    #Reviews Num
    reviews_Num=soup.find("span",{"class": "product-intro__head-reviews-text color-blue-text"})
    if(reviews_Num is not None):
        reviews_value=int(reviews_Num.get('data-review-num'))
    else:
        reviews_value=0
    
    
    #5 Star Images&Reviews Amountt
    five_reviews_value,five_images_value=get_ranks_level(driver,5)
    print(item_link)
    print(five_reviews_value,five_images_value)
    driver.get(item_link)
    #4 Star Images&Reviews Amount
    four_reviews_value,four_images_value=get_ranks_level(driver,4)
    print(four_reviews_value,four_images_value)
    driver.get(item_link)
    #3 Star Images&Reviews Amount
    three_reviews_value,three_images_value=get_ranks_level(driver,3)
    print(three_reviews_value,three_images_value)
    driver.get(item_link)
    #2 Star Images&Reviews Amount
    two_reviews_value,two_images_value=get_ranks_level(driver,2)
    print(two_reviews_value,two_images_value)
    driver.get(item_link)
    #1 Star Images&Reviews Amount
    one_reviews_value,one_images_value=get_ranks_level(driver,1)
    print(one_reviews_value,one_images_value)
    




    #Points Earnning
    points_earnning=soup.find("span",{"class": "color-orange-tips"})
    if(points_earnning is not None):
        points_earnning_text=points_earnning.text
        pattern=r"(\w+)"
        points_earnning_string=re.findall(pattern,points_earnning_text)[0]
        points_earnning_value=int(points_earnning_string)
    else:
        points_earnning_value=0
        
    #Description 
    for i in soup.findAll("div",{"class":"product-intro__description-table-item"}):
        current_key=i.find("div",{"class":"key"}).text[0:-1]
        current_val=i.find("div",{"class":"val"}).text
        description_dictionary[current_key]=current_val
        
  
    
    dictionary={'SKU':SKU_value,
                'Item Name':Item_Name_value,
                'Price':price_value,
                'Reviews Num':reviews_value,
                '1 Star Reviews':one_reviews_value,
                '1 Star Images':one_images_value,
                '2 Stars Reviews':two_reviews_value,
                '2 Stars Images':two_images_value,
                '3 Stars Reviews':three_reviews_value,
                '3 Stars Images':three_images_value,
                '4 Stars Reviews':four_reviews_value,
                '4 Stars Images':four_images_value,
                '5 Stars Reviews':five_reviews_value,
                '5 Stars Images':five_images_value,
                'Points Earnning':points_earnning_value,
               }
    dictionary.update(description_dictionary)
    return(dictionary)

In [44]:
def insert_details_to_dataframe(dataframe,details):
    print(details)
    dataframe=dataframe.append( pd.Series(details), ignore_index=True)
    return( dataframe)

In [45]:

def build_dataframe_rows(dataframe,all_products_links):
    i=0
    for link in all_products_links:
        i=i+1
        item_dictionary=get_details_from_single_item(link)
        dataframe=insert_details_to_dataframe(dataframe,item_dictionary)
        if((i > 99) and ((i % 100) == 0)):
            dataframe.to_csv("shein_{0}.csv".format(i))
    return (dataframe)

In [1]:
import time

def main():

    df=pd.DataFrame(columns = columns_list)
#     pages_amount=pages_amount()
    all_products_links=get_url_for_each_page(29)
    df=build_dataframe_rows(df,all_products_links)
    df.to_csv("shein_all.csv")
    
main()

In [170]:

driver.get("https://eur.shein.com/Drop-Shoulder-Zip-Up-Thermal-Drawstring-Hoodie-p-1609134-cat-1773.html?scici=productDetail~~RecommendList~~RS_emarsys,RJ_NoFaultTolerant~~Customers%20Also%20Viewed~~SPcProductDetailCustomersAlsoViewedList~~0~~0")
drop_down = driver.find_element_by_class_name('common-reviews__select-box-list')
drop_box = driver.find_element_by_class_name('common-reviews__select-box-input')
lines=drop_down.find_elements_by_tag_name('li')
hover_action = ActionChains(driver).move_to_element(drop_box)
hover_action.perform()
lines[5-5].click()
time.sleep(1)
images=driver.find_element_by_class_name("j-expose__review-image-tab-target")
images_text=images.text
pattern=r'Image (\(\d+\))'
images_value=int((re.findall(pattern,images_text)[0])[1:-1])
all_reviews=driver.find_element_by_class_name("j-expose__review-tab-container")
all_reviews_text=all_reviews.text
pattern=r'All Reviews (\(\w+)\+?\)'
all_reviews_value=int((re.findall(pattern,all_reviews_text)[0])[1:])
print(all_reviews_value)

2188


In [77]:
from selenium.webdriver.common.action_chains import ActionChains
driver.get('https://eur.shein.com/High-Waist-Argyle-Print-Wide-Leg-Jeans-p-1870870-cat-1934.html?scici=navbar_WomenHomePage~~tab01navbar04menu18~~4_18~~real_1930~~~~0~~50001')
drop_down = driver.find_element_by_class_name('common-reviews__select-box-list')
drop_box = driver.find_element_by_class_name('common-reviews__select-box-input')
# print(drop_down)
x=drop_down.find_elements_by_tag_name('li')
# print(x[4].is_enabled())
hover = ActionChains(driver).move_to_element(drop_box)
hover.perform()
x[0].click()